<a href="https://colab.research.google.com/github/Abdullokh3/AI_Application_final_exam/blob/main/week_13/Week_13_lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  9257k      0  0:00:08  0:00:08 --:--:-- 15.4M


In [ ]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random. Random( 1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname,
    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
  "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
  "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
  "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
from tensorflow.keras import layers
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)
int_val_ds = val_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)
int_test_ds = test_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)


In [ ]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32)) (embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid") (x)
model = keras.Model(inputs, outputs)
model. compile(optimizer="rmsprop",
  loss="binary_crossentropy",
  metrics=["accuracy"])
model. summary ()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               5128448   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
___________________________________________________

In [ ]:
callbacks = [
  keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]
model. fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")


Epoch 1/10
625/625 [==============================] - 170s 259ms/step - loss: 0.5339 - accuracy: 0.7499 - val_loss: 0.3831 - val_accuracy: 0.8604
Epoch 2/10
625/625 [==============================] - 166s 266ms/step - loss: 0.3692 - accuracy: 0.8669 - val_loss: 0.3069 - val_accuracy: 0.8854
Epoch 3/10
625/625 [==============================] - 166s 265ms/step - loss: 0.2777 - accuracy: 0.9047 - val_loss: 0.3245 - val_accuracy: 0.8910
Epoch 4/10
625/625 [==============================] - 166s 266ms/step - loss: 0.2367 - accuracy: 0.9185 - val_loss: 0.2916 - val_accuracy: 0.8924
Epoch 5/10
625/625 [==============================] - 167s 266ms/step - loss: 0.2007 - accuracy: 0.9324 - val_loss: 0.2800 - val_accuracy: 0.8904
Epoch 6/10
625/625 [==============================] - 169s 270ms/step - loss: 0.1767 - accuracy: 0.9420 - val_loss: 0.3153 - val_accuracy: 0.8954
Epoch 7/10
625/625 [==============================] - 168s 269ms/step - loss: 0.1572 - accuracy: 0.9499 - val_loss: 0.3942 -

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-11-30 09:49:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-30 09:49:37--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-30 09:49:37--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ")
    embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [ ]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get (word)
if embedding_vector is not None:
  embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers. Embedding(
  max_tokens,
  embedding_dim,
  embeddings_initializer=keras.initializers.Constant(embedding_matrix),
  trainable=False,
  mask_zero=True,
)